<a href="https://colab.research.google.com/github/sobti/NST/blob/main/Neural%20File%20Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
data=datasets.MNIST(root='data',download=True,transform=ToTensor())

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.76MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.06MB/s]


In [ ]:
data_loader=DataLoader(data,batch_size=64,shuffle=True)

In [ ]:
# learn tuple , set , dict


torch.Size([64, 1, 28, 28])


In [ ]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.sequential=nn.Sequential(
        nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU())
        self.sequential1=nn.Sequential(
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3),
        nn.BatchNorm2d(num_features=64),
        nn.ReLU())
        self.pool=nn.MaxPool2d(2)
        self.flatten=nn.Flatten()
        self.linear1=nn.Linear(in_features=64*5*5,out_features=10)
        self.softmax=nn.Softmax()
    def forward(self,x):
        x=self.sequential(x)
        x=self.pool(x)
        x=self.sequential1(x)
        x=self.pool(x)
        x=self.flatten(x)
        x=self.softmax(self.linear1(x))
        return x


In [ ]:
!pip install torchsummary
import torchsummary as torchsummary

In [ ]:
torchsummary.summary(model(),(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
              ReLU-3           [-1, 32, 26, 26]               0
         MaxPool2d-4           [-1, 32, 13, 13]               0
            Conv2d-5           [-1, 64, 11, 11]          18,496
       BatchNorm2d-6           [-1, 64, 11, 11]             128
              ReLU-7           [-1, 64, 11, 11]               0
         MaxPool2d-8             [-1, 64, 5, 5]               0
           Flatten-9                 [-1, 1600]               0
           Linear-10                   [-1, 10]          16,010
          Softmax-11                   [-1, 10]               0
Total params: 35,018
Trainable params: 35,018
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [ ]:
import torchvision
import torch as torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
model=torchvision.models.vgg19(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 82.5MB/s]


In [ ]:
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.select_features = ['0', '5', '10', '19', '28'] #conv layers
    self.vgg = torchvision.models.vgg19(pretrained=True).features
    print(self.vgg)

  def forward(self, output):
    features = []
    for name, layer in self.vgg._modules.items():
      output = layer(output)
      if name in self.select_features:
        features.append(output)
    return features

#load the model
vgg = VGG()

In [ ]:
class ContentLoss(nn.Module):
    def __init__(self, target):
        super().__init__()
        self.target = target.detach()

    def forward(self, x):
        self.loss = nn.functional.mse_loss(x, self.target)
        return x

In [ ]:
i=0
content_img=' '
new=nn.Sequential()
for layer in model.features.children():
    if isinstance(layer, nn.Conv2d):
        i += 1
        new.add_module(f"Cov2d_{i}", layer)
        target=new(content_img)
        loss=ContentLoss(target)
        new.add_module(f"content_loss_{i}", loss)


In [ ]:
vgg=torchvision.models.vgg19(pretrained=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
class content_loss(nn.Module):
    def __init__(self,target):
        super().__init__()
        self.target=target.detach()
    def forward(self,x):
        self.loss=nn.functional.mse_loss(x,self.target)
        return x

In [ ]:
def myfunc():
 new_model=nn.Sequential()
 i=0
 content_img=torch.randn(1,3,224,224)
 for layer in vgg.features:
    if isinstance(layer,nn.Conv2d):
        i += 1
        name=f"conv_{i}"
        new_model.add_module(name,layer)
    elif isinstance(layer,nn.ReLU):
        name=f"relu_{i}"
        new_model.add_module(name,nn.ReLU())
    elif isinstance(layer,nn.MaxPool2d):
        name=f"maxpool_{i}"
        new_model.add_module(name,layer)
    else: continue

    if  name in ['conv_1','conv_2','conv_3','conv_4','conv_5']:
        target=new_model(content_img)
        loss=content_loss(target)
        new_model.add_module(f"content_loss_{i}",loss)
 return new_model








In [ ]:
model=myfunc()

In [ ]:
model(torch.randn(1,3,224,224))

In [3]:
import torch
import torchvision
from torchvision import models
from torch import nn
import numpy
import torch.optim as optimization
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
# prompt: open file from "/content/style_image.jpg"

from PIL import Image







In [ ]:
style_img = Image.open("/content/style.jpg")
transform_image=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize((0.6971, 0.7078, 0.6860),(0.2333, 0.2431, 0.3146))])
style_image=transform_image(style_img)
content_img=transform_image(content_img)
target_img = content_img.clone().requires_grad_(True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
vgg=torchvision.models.vgg19(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg=vgg.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
def content_loss(x,target):
        loss=nn.functional.mse_loss(x,target)
        return loss

In [19]:
def gram_matrix(x):
        a,b,c,d=x.size()
        x=x.view(a*b,c*d)
        gram=torch.mm(x,x.t())
        return gram/b*c*d

In [8]:
def style_loss(x,target):
        loss=nn.functional.mse_loss(gram_matrix(x),gram_matrix(target))

        return loss


In [9]:
content_layers=['conv_1','conv_5','conv_10','conv_15','conv_20']
style_layers=['conv_1','conv_5','conv_10','conv_15','conv_20']

In [45]:
def get_model_and_loss(cnn):
        cnn = cnn.eval()
        new_model=nn.Sequential()
        i=0
        for layer in cnn.features.children():
            if isinstance(layer,nn.Conv2d):
                i += 1
                name=f"conv_{i}"
                new_model.add_module(name,layer)
            elif isinstance(layer,nn.ReLU):
                name=f"relu_{i}"
                new_model.add_module(name,nn.ReLU())
            elif isinstance(layer,nn.MaxPool2d):
                name=f"maxpool_{i}"
                new_model.add_module(name,layer)
            else: continue

        return new_model





In [46]:
model=get_model_and_loss(vgg).to(device)

In [47]:
def get_features(output):
    content_features=[]
    for layer in model.named_children():
        output=layer[1](output)
        if layer[0] in content_layers:
            content_features.append(output)
    return content_features

In [15]:
import torch
from tqdm import tqdm

In [73]:
def style_transfer(content_img):
  target_feature=[]
  style_img = Image.open("/content/style.jpeg")
  transform_image=transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])

  style_image=transform_image(style_img)

  content_img=transform_image(content_img)

  target_img = content_img.clone().requires_grad_(True)

  optimizer=optimization.Adam([target_img],lr=0.003)

  content_feature = []

  style_feature = []



  for step in tqdm(range(3000)):
   #get feature vectors representations for every image
   content_feature = get_features(content_img.unsqueeze(0).to('cuda'))
   style_feature = get_features(style_image.unsqueeze(0).to('cuda'))
   target_feature = get_features(target_img.unsqueeze(0).to('cuda'))


   #initiate the losses
   current_style_loss = 0
   current_content_loss = 0

   for target, content, style in zip(target_feature, content_feature, style_feature):
     current_content_loss += content_loss(target, content)
     current_style_loss += style_loss(target, style)

   #calculate the total loss according to the original paper
   total_loss = current_content_loss+10000000*current_style_loss

   #set parameters to zero
   optimizer.zero_grad()
   #compute the gradient
   total_loss.backward()
   #update parameters
   optimizer.step()
   #uncomment if you want to follow the progress
   #if step%2 == 0:
     #print(f'step: {step}, content loss: {current_content_loss}, style loss: {current_style_loss}')
  return convert_image(target_img)


In [64]:
from torchvision.transforms.functional import to_pil_image
def convert_image(img):


# Ensure tensor is on CPU and detached from computation graph
 tensor = img.detach().cpu()

 image = to_pil_image(tensor)
 return image

In [20]:
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00


In [74]:
gr.Interface(
    fn=style_transfer,
    inputs=[
        gr.Image(type="pil", label="Content Image"),
    ],
    outputs=gr.Image(label="Stylized Output"),
    title="Neural Style Transfer",
    description="Upload a content image and a style image. The model will blend the content with the style."
).launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c26cb27848206e4ae9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


100%|██████████| 3000/3000 [15:58<00:00,  3.13it/s]


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://c26cb27848206e4ae9.gradio.live
